In [25]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from fedland.utils import load_all_training_results, get_experiment_description, plot_results_overview, plot_client_info
from sklearn.linear_model import LinearRegression

## Analyze Generalizability
Things to try:
- Validation Acc alone and (train acc - test acc)
- Validation Loss and (train loss - test loss)

In [26]:
df = load_all_training_results(results_path="../results")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   index                        15320 non-null  int64         
 1   epoch                        15320 non-null  int64         
 2   learning_rate                15320 non-null  float64       
 3   batch_size                   15320 non-null  int64         
 4   len_train_indices            15320 non-null  int64         
 5   len_test_indices             15320 non-null  int64         
 6   momentum                     15320 non-null  float64       
 7   train_loss                   15320 non-null  float64       
 8   train_accuracy               15320 non-null  float64       
 9   test_loss                    15320 non-null  float64       
 10  test_accuracy                15320 non-null  float64       
 11  path_norm                    15320 non-nu

In [27]:
# df_last_epoch = df.loc[df.groupby('experiment_id')['epoch'].idxmax()]
df_train = df.copy()
df_train = df_train[df_train.groupby('experiment_id')['epoch'].transform('max') == df_train['epoch']]
df_train["pct_change_path_norm_round"] = (
    (df_train["path_norm"] - df_train["global_path_norm"]) / df_train["global_path_norm"].abs()
)

In [29]:
x_vars = ["pct_change_path_norm_round", "experiment_id", "client_index", "len_train_indices"]
Y = "test_accuracy"
df_train['experiment_id'] = df_train['experiment_id'].astype('category')
df_train['client_index'] = df_train['client_index'].astype('category')

X = df_train[x_vars]
y = df_train[Y]

model = LinearRegression()
model.fit(X, y)
print("Model Coefficients:")
for var, coef in zip(x_vars, model.coef_):
    print(f"{var}: {coef:.4f}")

print(f"Intercept: {model.intercept_:.4f}")

ValueError: could not convert string to float: '3f6c270c-ff3d-4c1d-a01c-3359312cd3c4'